In [1]:
import pandas as pd
import numpy as np

import json
import requests
from IPython.display import clear_output

import psycopg2 as pg
conn = pg.connect(database = "discoverify_db",
                  user="postgres", 
                  password="damara1004")

conn.autocommit = True

def run_query(q):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(q)
            return pd.read_sql(q, conn)

        except (Exception, pg.DatabaseError) as error:
            print(error)

def run_command(c):
    with conn:
        try:
            cur = conn.cursor()
            cur.execute(c)
            cur.close()
            conn.commit()
            
        except (Exception, pg.DatabaseError) as error:
            print(error)
            
# reading in created datasets
# track_metadata = pd.read_csv('../data/all_tracks_cleaned_V2.csv', index_col=0)
# spotify_features = pd.read_csv('../data/audio_features_final.csv', index_col=0)

In [6]:
c = '''
    CREATE DATABASE discoverify_db;
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS metadata (
        track_id VARCHAR(22) PRIMARY KEY,
        track_name TEXT NOT NULL,
        artist TEXT NOT NULL,
        artist_id VARCHAR(22) NOT NULL,
        genre_1 TEXT NOT NULL,
        genre_2 TEXT,
        genre_3 TEXT
        );
        
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS spotify_features (
        id VARCHAR(22) PRIMARY KEY,
        danceability NUMERIC,
        energy NUMERIC,
        loudness NUMERIC,
        speechiness NUMERIC,
        acousticness NUMERIC,
        instrumentalness NUMERIC,
        liveness NUMERIC,
        valence NUMERIC,
        tempo NUMERIC,
        
        FOREIGN KEY (id) REFERENCES metadata (track_id)
        );
    '''

run_command(c)

c = '''
    CREATE TABLE IF NOT EXISTS librosa_features (
        track_id VARCHAR(22) PRIMARY KEY,
        spectral_centroid NUMERIC,
        spectral_bandwidth NUMERIC,
        rolloff NUMERIC,
        zero_crossing_rate NUMERIC,
        mfcc1 NUMERIC,
        mfcc2 NUMERIC,
        mfcc3 NUMERIC,
        mfcc4 NUMERIC,
        mfcc5 NUMERIC,
        mfcc6 NUMERIC,
        mfcc7 NUMERIC,
        mfcc8 NUMERIC,
        mfcc9 NUMERIC,
        mfcc10 NUMERIC,
        mfcc11 NUMERIC,
        mfcc12 NUMERIC,
        mfcc13 NUMERIC,
        mfcc14 NUMERIC,
        mfcc15 NUMERIC,
        mfcc16 NUMERIC,
        mfcc17 NUMERIC,
        mfcc18 NUMERIC,
        mfcc19 NUMERIC,
        mfcc20 NUMERIC,
        C NUMERIC,
        "C#" NUMERIC,
        D NUMERIC,
        "D#" NUMERIC,
        E NUMERIC,
        F NUMERIC,
        "F#" NUMERIC,
        G NUMERIC,
        "G#" NUMERIC,
        A NUMERIC,
        "A#" NUMERIC,
        B NUMERIC,

        FOREIGN KEY (track_id) REFERENCES metadata (track_id)
        );
'''

run_command(c)

### Inserting Track Metadata

In [15]:
metadata_cols = ['track_id','track_name','artist','artist_id','genre_1','genre_2','genre_3']

metadata = track_metadata[metadata_cols]

In [4]:
metadata = pd.read_csv('../data/metadata_cleaned.csv', sep=";")
metadata.head()

,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,art pop,chamber pop,indie pop
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk


In [5]:
tuple(metadata.iloc[28,:].values)

('7oJmIBxEAYjfkS7lhZ7M1S',
 'Sometimes',
 'Goth Babe',
 '7o96HO2zrujyATtVsqGhh3',
 'indie garage rock',
 'indie pop',
 'modern rock')

In [61]:
for i in range(20,30):
    record = metadata.iloc[i,:].values
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
        
    
    print(new_record)

('2CPLGREl5ahzA2WPBMkpxh', 'Fallingwater', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('4HfLQJtVT1KiX1eVedDyTm', 'Alaska', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('68vxG1i6RSmrsY9mRdqDgG', 'Burning', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('6PyZGb1rB7oLKCICWTmGYa', 'Say It', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('3D4uUFRoYDFHbkQlTKL2Om', 'Give A Little', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('5sn1BmEOPP1bu4cZq9OgoF', 'Tim McGraw - Recorded At Spotify Studios NYC', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('4L9l5x0WuzNtyEBeqzvgPE', 'Overnight', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('5RRNZFyOi17nTh2bPEKPtp', 'Dog Years', 'Maggie Rogers', '4NZvixzsSefsNiIqXn0NDe', 'electropop', 'pop', None)
('7oJmIBxEAYjfkS7lhZ7M1S', 'Sometimes', 'Goth Babe', '7o96HO2zrujyATtVsqGhh3', 'indie 

In [24]:
cur = conn.cursor()

err = []
for i in range(1101,len(metadata)):
    record = tuple(metadata.iloc[i,:].values)
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
    
    try:
        cur.execute("INSERT INTO metadata (track_id, track_name, artist, artist_id, genre_1, genre_2, genre_3) VALUES (%s, %s, %s, %s, %s, %s, %s)", record)
        conn.commit()
        
    except:
        err.append(new_record)
        pass

In [25]:
q = '''SELECT COUNT(*) FROM metadata'''

df = run_query(q)
df

,count
0,190094


### Inserting Spotify Features

In [35]:
spotify_features = pd.read_csv('../data/spotify_features.csv')
spotify_features.head()

,id,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0Tbgn7Ocm54WoqJUsIlfp4,0.644,0.912,-6.072,0.0384,0.00301,0.657000,0.0782,0.222,126.011
1,1XibR7VrDllfBz6yuTndmb,0.696,0.623,-7.600,0.1260,0.01750,0.000091,0.1430,0.573,173.951
2,3tjFYV6RSFtuktYl3ZtYcq,0.700,0.722,-3.558,0.0369,0.22100,0.000000,0.2720,0.756,90.989
3,27u7t9d7ZQoyjsCROHuZJ3,0.779,0.705,-3.895,0.0344,0.36900,0.000008,0.1240,0.946,101.022
4,6piAUJJQFD8oHDUr0b7l7q,0.747,0.717,-4.841,0.1790,0.19900,0.000000,0.1320,0.523,150.965


In [36]:
len(spotify_features)

200006

In [37]:
# generate list of columns for insert statement

for c in spotify_features.columns:
    print(f"{c},")

id,
danceability,
energy,
loudness,
speechiness,
acousticness,
instrumentalness,
liveness,
valence,
tempo,


In [40]:
cur = conn.cursor()

err = []
for i in range(10, len(spotify_features)):
    record = tuple(spotify_features.iloc[i,:].values)
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
    
    try:
        cur.execute("""INSERT INTO spotify_features (id, danceability, energy, loudness, 
                                                     speechiness, acousticness, instrumentalness, 
                                                     liveness, valence, tempo) 
                                   VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", record)
        conn.commit()
        
    except:
        err.append(new_record)
        pass

In [39]:
err

[]

### Inserting Librosa Features

In [42]:
# librosa_features = pd.read_csv('../data/librosa_features.csv')
# print(len(librosa_features))

# generate list of columns for insert statement
for c in librosa_features.columns:
    print("%s,")

%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,
%s,


In [62]:
cur = conn.cursor()

err = []
for i in range(20, len(librosa_features)):
    record = tuple(librosa_features.iloc[i,:].values)
    
    new_record = tuple(elem if str(elem)!='nan' else None for elem in record)
    
    try:
    
        cur.execute("""INSERT INTO librosa_features (track_id, spectral_centroid, spectral_bandwidth, rolloff, zero_crossing_rate,
                                                     mfcc1, mfcc2, mfcc3, mfcc4, mfcc5, mfcc6, mfcc7, mfcc8, mfcc9, mfcc10, 
                                                     mfcc11, mfcc12, mfcc13, mfcc14, mfcc15, mfcc16, mfcc17, mfcc18, mfcc19, 
                                                     mfcc20, C, "C#", D, "D#", E, F, "F#", G, "G#", A, "A#", B)                                    
                                   VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,
                                           %s,%s,%s,%s,%s,%s,%s,%s,%s)""", new_record)
        conn.commit()
        
    except:
        err.append(new_record)
        pass
        

In [69]:
missing_records = [e[0] for e in err]
missing_records

['04hkFRl2HK1XHimX3rDTLd',
 '07XhZyYKgZMT4SBAddwVhl',
 '0AYRvBiAJsjql6awg9HAMA',
 '0cknshtbPrUKRSZTkBNbRT',
 '0kzGjSRHGVcaWxagzQwAyD',
 '0L906KB21EDCWVhveBemQo',
 '0NIzmplbLg4ecijHZO3CpL',
 '0nLoU2yK9bb8JMj9RGhzZI',
 '0olZcym6fd7jhsQZwMzfzf',
 '0XoGYo00bbURIc7ythsj2J',
 '0z3FgElQdeuYxAyEj5PEqQ',
 '0ZiETcI1FKuIaOxB2h2HUj',
 '0zOHlap4WQgkf4znDtGzNp',
 '129vNubdiTakGntg0fSbaw',
 '12rlcObB9ztGh7CPmY1g4u',
 '17KeHeGWyRNkbGnZxGBPl7',
 '17q3MUiNDnzC8fNQfGrQ4r',
 '1B18oU93wYoaR4nLDi5EKc',
 '1bJZQZ0hYcOt1ZhwwvTMtx',
 '1egRt71E8ssYxdBWpQqaaE',
 '1fy3c3UMSkiXdATN9ChwK6',
 '1IAFf9zkKzLwyUn7h63dmC',
 '1jhclVcqFpy4IGdWYzn9QS',
 '1kK9i5e6Y6ifSIYAH82xoL',
 '1oRy2CCBaWCVqJRkmyGWLx',
 '1poMJGPzlqJWK5jgH7USQO',
 '1Upi1oHB1ly7XdlbOz7Gjd',
 '1Upvn947hYjd7VTQghwOHM',
 '1WAoyUh9QgkSCdEeKBt4S1',
 '1WCsEbnaJ0yKNQCETD2ruC',
 '1XlrJfxjejyN0SipJK8epC',
 '27ptpU9FIFGty7GuSmitnc',
 '2dsXGvlg3gSSpZiJSQks3g',
 '2H2379xRxgQVvcOSoL8Ajr',
 '2l8wbBST63jijlyDtVTAPi',
 '2NRhy2p5nQkv62kaec3BW6',
 '2O0ZxPTJLVj1R8dZ0IjiM7',
 

In [73]:
librosa_features[librosa_features['track_id'].startswith'6OZbBfQnhI0y7ePuFQwPA8']

KeyError: False

In [77]:
metadata[metadata['track_id'].str.startswith('6OZ')==True]

,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
22356,6OZcnhQwwtDJNkOBzgF50r,Do You Think We’ll Last Forever?,Caroline Rose,06W84OT2eFUNVwG85UsxJw,art pop,indie folk,indie rock
124998,6OZYfkgwUiKrIz6F5vW5oL,Where Are Ü Now (with Justin Bieber) - Rustie ...,Jack Ü,1HxJeLhIuegM3KgvPn8sTa,edm,electro house,NaN
134293,6OZLrNbCCtGM596YZJ8deM,Susan's Gone,Dirk Powell,0WLXcCZ84s9zsmIOoJeEwG,appalachian folk,banjo,bluegrass
149140,6OZ210lsvCsoIqjBZcGYvm,Out of Nowhere,Django Reinhardt,5Z1XZyEFY0dewG8faEIiEx,bebop,cool jazz,french jazz


In [81]:
old_metadata = pd.read_csv('../data/metadata.csv', sep=";")
old_metadata = old_metadata[old_metadata.columns[2:]]
old_metadata

,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
0,6YDf6QV7QfCEo8O2dbWalK,Real Love Baby,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
1,0qaWxVVd3KrP4WY27KWpIe,In a River,Rostam,04XggbrM51GcFPTxBYtRXT,art pop,chamber pop,indie pop
2,1yZMv2GMAibgLGsQiowZrt,Nancy From Now On,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
3,2eg2gvPXuwZ9FyrPaLgrXi,Chateau Lobby #4 (in C for Two Virgins),Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
4,0wzruvvN7f5wu39aFcjTMw,Hollywood Forever Cemetery Sings,Father John Misty,2kGBy2WHvF0VdZyqiVCkDT,art pop,chamber pop,freak folk
...,...,...,...,...,...,...,...
230451,6jVVnTIC1zHFFdn2rBOE2g,This Must Be the Place (Naive Melody) - Live,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230452,3xJ8siSD7r5mDqLNtQ5CPf,A Soft Seduction,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230453,6Mc0SvjQ2By4ZJWdF77M2C,Dance on Vaseline,David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock
230454,392cd1euczCOcZAs0SvvZS,The Last Emperor (Main Title Theme),David Byrne,20vuBdFblWUo2FCOvUzusB,art pop,art punk,art rock


In [82]:
old_metadata[old_metadata['track_id'].isin(missing_records)]

,track_id,track_name,artist,artist_id,genre_1,genre_2,genre_3
31517,2oAT0SSlDZjHdp4yWsVEUX,Jungle Queen,LP Giobbi,3oKnyRhYWzNsTiss5n4Z1J,NaN,NaN,NaN
31519,1jhclVcqFpy4IGdWYzn9QS,Gas Me Up,LP Giobbi,3oKnyRhYWzNsTiss5n4Z1J,NaN,NaN,NaN
36196,4tWW9Zol6NbI3cl0Jj8Ipy,Let U Go,Unghetto Mathieu,2crfTbTl8cTxLFidTcd2Tg,NaN,NaN,NaN
36197,3FfWjwjwjVDZWlddoQ7jP9,23,Unghetto Mathieu,2crfTbTl8cTxLFidTcd2Tg,NaN,NaN,NaN
36200,3VpHbYx7GtOVLnl2C8KhDr,Karma,Unghetto Mathieu,2crfTbTl8cTxLFidTcd2Tg,NaN,NaN,NaN
...,...,...,...,...,...,...,...
73658,744eg02EmdTJzUdOqzybii,Dancing Queen - Sondr Remix,Daecolm,1IFAU4mznUcfPVP9z2c24N,NaN,NaN,NaN
74613,3Hi1zGgdpj095HCIuZNe4Y,Collaboration,Evelyn Ida Morris,7IWZBV2hVnVn1yRjnGp6uB,NaN,NaN,NaN
74967,2H2379xRxgQVvcOSoL8Ajr,Somebody's Watching Me,Rockwell,3xs0LEzcPXtgNfMNcHzLIP,NaN,NaN,NaN
74969,2p2mp65C5lzOvHGRwEEtXu,Somebody's Watching Me - 80s Flashback Dance F...,Rockwell,3xs0LEzcPXtgNfMNcHzLIP,NaN,NaN,NaN


In [83]:
len(librosa_features)

46721

In [84]:
q = '''SELECT COUNT(*) FROM metadata'''

d = run_query(q)
d

,count
0,190095


In [85]:
q = '''SELECT COUNT(*) FROM spotify_features'''

d = run_query(q)
d

,count
0,159647


In [86]:
q = '''SELECT COUNT(*) FROM librosa_features'''

d = run_query(q)
d

,count
0,46605


It seems that the records that weren't inserted into the `librosa_features` table don't have a genre. So its fine that they weren't inserted since I can't use them anyway.

__REMINDERS__: 

* Don't forget to change 'NaN' values in each table to nulls.
* Before running librosa pipeline again, cross-reference for tracks that are missing key information so I'm not wasting time generating feature vectors for songs that I can't use.